# Trabajo practico NLP
## Sistema LLM RAG
El sistema RAG toma datos de dos fuentes de datos distintas, una base de datos vectorial y otra de grafos.

La información que proporciona el sistema es sobre:

- Expertos, peritos o consultores en materia ambiental.
- Reservas naturales en provincias argentinas.
- Animales en peligro de extinción en Argentina.

## Datos utilizados:
### - Base de datos vectorial
Los datos almacenados en la base de datos vectorial constan de un libro de más de 500 páginas que habla sobre especies en peligro de extinción en Argentina y datos tabulares provenientes de un archivo CSV (descargado de la página https://datos.gob.ar/) que trata sobre consultores, peritos y expertos en materia ambiental.

### - Base de datos en grafos
Los datos almacenados en la base de datos en grafos tratan sobre las reservas naturales de las provincias argentinas, conteniendo información del año de creación, cantidad de hectáreas que preservan, ambiente que protegen, entre otros datos.

## Instalar e importar librerias

In [ ]:
#---------# instalacion de librerias #---------#
# instalar librerias de llama_index
!pip install llama_index
!pip install llama-index-embeddings-langchain

# instalar librerias de langchain
!pip install langchain
!pip install langchain-community[chroma]
!pip install langchain-community[sentence_transformer]
!pip install langchain_openai

# instalar librerias de sentence-transformers
!pip install sentence-transformers

# instalar libreria de chromadb
!pip install chromadb

# instalar libreria de base de datos en grafos
!pip install neo4j

#---------# importacion de libreria #---------#
# importar librerias para descargar carpeta con archivos
import gdown
import os
import shutil

# importar librerias para realizar lectura de datos y split
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
import tiktoken

# importar librerias para base de datos chromadb y modelo de Embeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
import getpass

# importar librerias para base de datos en grafos
import re
from neo4j import GraphDatabase

# importar libreria para modelos OpenAI
from openai import OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Definir variables de entorno

In [ ]:
# Pedir al usuario que ingrese la API KEY de openAI
os.environ["OPENAI_API_KEY"] = input("Ingrese la API KEY de openAI: ")

# Pedir al usuario que ingrese la API KEY de hugginsface
os.environ["inference_api_key"] = input("Ingrese la API KEY de hugginsface: ")

# descargar tus archivos desde una carpta de drive
os.environ["url"] = 'https://drive.google.com/drive/folders/1uzjlNir2K6CJj2krjn5ifHKjXhc8Af2C?usp=sharing'


## Descargar archivos

In [ ]:
# Link con archivos sobre historia Argentina
url =  os.environ.get("url")

# Descarga carpeta
gdown.download_folder(url, quiet=True, output='prueba')

# Crear la carpeta 'llamaindex_data' si no existe
carpeta_destino = 'llamaindex_data'
if not os.path.exists(carpeta_destino):
    os.makedirs(carpeta_destino)

# Mover todos los archivos a 'llamaindex_data'
carpeta_origen = 'prueba'
for filename in os.listdir(carpeta_origen):
    ruta_origen = os.path.join(carpeta_origen, filename)
    ruta_destino = os.path.join(carpeta_destino, filename)
    shutil.move(ruta_origen, ruta_destino)

# Eliminar la carpeta de drive
shutil.rmtree(carpeta_origen)

print("Archivos movidos con éxito.")

Archivos movidos con éxito.


## Realizar carga de informacion y split de texto

In [ ]:
data = []
# Carpeta donde se encuentran los archivos PDF
carpeta = '/content/llamaindex_data'

# Itera sobre los archivos en la carpeta
for nombre_archivo in os.listdir(carpeta):
    print(nombre_archivo)
    if nombre_archivo.endswith('.pdf'):  # Asegúrate de que solo estás leyendo archivos PDF
        ruta_archivo = os.path.join(carpeta, nombre_archivo)
        loader = PyPDFLoader(ruta_archivo)
        data = loader.load()

    elif nombre_archivo.endswith('.csv'):
        ruta_archivo = os.path.join(carpeta, nombre_archivo)
        loader = CSVLoader(ruta_archivo, encoding="utf-8")
        data.extend(loader.load())
    else:
      continue



from langchain.text_splitter import CharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=100
)
texts = text_splitter.split_documents(data)


encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

for i in range(5):
    print("El documento tiene %s tokens" % len(encoding.encode(texts[i].page_content)))

for i in range(5):
  print(texts[i].page_content)

especies-amenazadas-de-la-argentina.pdf
registro-consultores-expertos_1.csv
El documento tiene 61 tokens
El documento tiene 63 tokens
El documento tiene 380 tokens
El documento tiene 421 tokens
El documento tiene 491 tokens
DOS DÉCADAS DE TRABAJO CONEDITORES: ANALÍA V. DALIA, VALERIA BAUNI, MARINA HOMBERG Y ADRIÁN GIACCHINO
ESPECIES AMENAZADAS
DE    LA ARGENTINA
DOS DÉCADAS DE TRABAJO CON
ESPECIES AMENAZADAS
DE LA ARGENTINA
EDITORES: 
ANALÍA V. DALIA, VALERIA BAUNI, MARINA HOMBERG Y ADRIÁN GIACCHINO
Diseño gráfico: Mariano Masariche.
Fundación de Historia Natural Félix de Azara
Centro de Ciencias Naturales, Ambientales y AntropológicasUniversidad Maimónides Hidalgo 775 P. 7º - Ciudad Autónoma de Buenos Aires.(54) 11-4905-1100 int. 1228  / www.fundacionazara.org.ar
Se ha hecho el depósito que marca la ley 11.723. No se permite la reproducción parcial o total, el almacenamiento, el 
alquiler, la transmisión o la transformación de este libro, en cualquier forma o por cualquier medio, sea 

## Crear embeddings y vectorstore

In [ ]:
inference_api_key = os.environ.get("inference_api_key")

embedding_function = HuggingFaceInferenceAPIEmbeddings(
    api_key=inference_api_key, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
ids_textos = [f"id:{i}" for i in range(1, len(texts)+1)]

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=texts,
    collection_name="vectores-chroma",
    embedding=embedding_function,
    ids=ids_textos
)


retriever = vectorstore.as_retriever()


In [ ]:
query = "cual es el estado de conservacion de la mojarra desnuda?"
retriever.get_relevant_documents(query)[0]

Document(page_content='94\nDOS DÉCADAS DE TRABAJO CON ESPECIES AMENAZADAS DE LA ARGENTINA.apartadas que eran dominio exclusivo de \nla mojarra desnuda (Quiroga, 2019). Ch. interruptus es un carácido cercano a la mojarra desnuda en términos evolutivos (Ringuelet, 1975; Cazzaniga, 1978; Almi-rón et al., 1997), por lo que sus requeri-\nmientos ecológicos resultarían similares y esto puede explicar en parte, la rápida dispersión y colonización observada en la cabecera del arroyo. Estudios previos indican que en sitios de característi-cas semejantes a las observadas para el arroyo Valcheta, la mojarra plateada posee una dieta similar a la del carácido desnudo (Escalante y Menni, 1999), por lo que la competencia por los recursos es altamente probable (Pérez et al., 2015). Según comentarios de acuaristas, la mojarra plateada puede alimentarse de huevos de otros peces, por lo que exis -\ntiría la posibilidad de que este pez con-suma las puestas de la mojarra desnuda, e incluso las de la rana d

In [ ]:
query = "¿cuantos expertos, peritos o consultores hay en la provincia de Santa Fe?"
retriever.get_relevant_documents(query)[0]

Document(page_content='numero_de_certificado: 143\napellido: Don\nnombre: Danisa\nprovincia: Santa Fe\nlocalidad: Santa Fe\ncorreo_electronico: 143_santafe@gmail.com\ntitulo_universitario: Ingeniería Ambiental', metadata={'row': 486, 'source': '/content/llamaindex_data/registro-consultores-expertos_1.csv'})

# Definir funciones para funcionamiento del RAG

## Consulta a base de datos de grafos

In [ ]:
def grafos(provincia_nombre):
    import re
    from neo4j import GraphDatabase

    URI = "neo4j+s://748c956d.databases.neo4j.io:7687"
    AUTH = ("neo4j", "_8VJN_774ncKZVZmiyQwuyfIc_mURbxRIrokcYIqXrU")

    # Conecta a la base de datos
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
      # Verifica la conexión
      driver.verify_connectivity()


    nombre = f"'{provincia_nombre}'"

    provincia = "{provincia: "

    provincia = provincia + nombre + '}'

    records, summary, keys = driver.execute_query(
        f"MATCH p= (:provincia {provincia})-[:`contiene reserva`]->() RETURN p LIMIT 25",
        )

    completo = ''

    for i in range(len(records)):

      # Extraer la información del diccionario
      provincia = records[i].data()['p'][0]['provincia']
      info_reserva = records[i].data()['p'][2]

      # Formatear la información en un string
      resultado_str = f"Provincia: {provincia} Información de la reserva: {info_reserva}"
      completo = completo + resultado_str
      cadena_limpia = re.sub(r'[{}\[\]\\\']', '', completo)

    return cadena_limpia

#completo = funcion_prueba("Chubut")
#print(completo)

## Clasificacion de preguntas

In [ ]:
def clasificador(pregunta):
  client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

  # Llamada a la API de OpenAI para obtener la respuesta
  chat_completion = client.chat.completions.create(
    messages = [
                {"role": "system", "content": """Eres un asistente que se encarga de identificar la fuente correcta de datos
    Debes responder obligatoriamente de la siguiente manera sin agregar información extra o de conocimiento previo:\n
    (vectores,) si la pregunta es sobre: peritos, consultores, expertos o sobre el estado de conservación de animales en Argentina.\n
    (grafos, [nombre de provincia argentina]) si la pregunta trata sobre reservas naturales, parques nacionales o ambientes que están en una provincia argentina. \n"""},

                {"role": "user", "content": "Cual es el estado de conservacion de la mojarra?"},
                {"role": "assistant", "content": "(vectores,)"},

                {"role": "user", "content": "Cuales son las reservas ecologicas de Chubut?"},
                {"role": "assistant", "content": "(grafos, Chubut"},

                {"role": "user", "content": "Cuantas reservas ecologicas hay en Buenos Aires?"},
                {"role": "assistant", "content": "(grafos, Buenos Aires)"},

                {"role": "user", "content": "Cuales son los ambientes protegidos de Santa Fe?"},
                {"role": "assistant", "content": "(grafos, Santa Fe)"},

                {"role": "user", "content": "cuantos consultores expertos hay en Cordoba?"},
                {"role": "assistant", "content": "(vectores,)"},

                {"role": "user", "content": "Cuantos animales hay en peligro de extincion en Argentina?"},
                {"role": "assistant", "content": "(vectores,)"},

                {"role": "user", "content": "porque esta en peligro de extincion el mono aullador rojo?"},
                {"role": "assistant", "content": "(vectores,)"},

                {"role": "user", "content": "cual es el correo electronico de un experto en ingenieria ambiental de Buenos Aires"},
                {"role": "assistant", "content": "(vectores,)"},

                {"role": "user", "content": "cuantos consultores expertos en biologia marina hay en Santa Fe"},
                {"role": "assistant", "content": "(vectores,)"},

                {"role": "user", "content":pregunta}
    ],
    model = "gpt-3.5-turbo-0125",
    temperature=0,
    max_tokens=50
  )
  respuesta = chat_completion.choices[0].message.content
  # Elimina los paréntesis y divide la cadena en una lista usando la coma como separador
  respuesta = respuesta.strip('()').split(', ')

  # Convierte la lista en una tupla
  tupla_resultante = tuple(respuesta)

  return tupla_resultante

#respuesta = clasificador(pregunta)
# Imprime la respuesta generada por el modelo
#print(respuesta)



In [ ]:
"""
preguntas = ["Cuantas reservas ecologicas hay en Tucuman?", "El mono aullador rojo esta extinto?", "Cuantos ambientes se protegen en la provincia de Santa Fe", "cual es el contacto de un experto en ingeniria ambiental de Buenos Aires" ]
for i in preguntas:
  respuesta = clasificador(i)
  print(respuesta)
"""


'\npreguntas = ["Cuantas reservas ecologicas hay en Tucuman?", "El mono aullador rojo esta extinto?", "Cuantos ambientes se protegen en la provincia de Santa Fe", "cual es el contacto de un experto en ingeniria ambiental de Buenos Aires" ]\nfor i in preguntas:\n  respuesta = clasificador(i)\n  print(respuesta)\n'

## Obtencion de contextos

In [ ]:
def obtener_contexto(respuesta, query):
  contexto = None
  if respuesta[0] == 'grafos':
    contexto = grafos(respuesta[1])
    return (contexto, "Base de datos grafos")

  elif respuesta[0] == 'vectores,':
      contexto = retriever.get_relevant_documents(query)[0]
      contexto = contexto.page_content
      # Obtener el cliente Chroma
      return (contexto, "Base de datos vectorial")
  else:
    print('Error al identificar la fuente')

## Definir LLM para responder preguntas

In [ ]:
def LLM(pregunta, contexto):
  client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

  # Llamada a la API de OpenAI para obtener la respuesta
  chat_completion = client.chat.completions.create(
    messages = [
                {"role": "system", "content": f'Eres un asistente que responde preguntas en base a la informacion de contexto proporcionada. No puedes utilizar conocimientos previos para responder. El contexto es el siguiente: {contexto}'},

                {"role": "user", "content":pregunta}

    ],
    model = "gpt-3.5-turbo-0125",
    temperature=1,
    max_tokens=300
  )
  respuesta = chat_completion.choices[0].message.content
  return respuesta



## Generar funcion RAG con formato de respuesta

In [ ]:
def sistema(pregunta):
   tupla = clasificador(pregunta)
   tupla_contexto = obtener_contexto(tupla, pregunta)
   contenido = tupla_contexto[0]
   respuesta = LLM(pregunta, contenido)
   fuente = tupla_contexto[1]
   contexto = contenido
   return (respuesta, fuente, contexto)


In [ ]:
def responder(prompt):
    # Llamar a la función del RAG
    respuesta, fuente, contexto = sistema(prompt)

    # Imprimir el resultado
    print('------------------------------------------------------------------------')
    print(f'PREGUNTA:\n{prompt}')
    print('------------------------------------------------------------------------')
    print(f'FUENTE ESCOGIDA: {fuente}')
    print('------------------------------------------------------------------------')
    print(f'CONTEXTO BRINDADO:\n{contexto}')
    print('------------------------------------------------------------------------')
    print(f'RESPUESTA:\n{respuesta}')

# Preguntas

In [ ]:
# Prompt:
prompt = '¿la lagartija de las dunas esta en peligro de extincion?'

responder(prompt)


------------------------------------------------------------------------
PREGUNTA:
¿la lagartija de las dunas esta en peligro de extincion?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos vectorial
------------------------------------------------------------------------
CONTEXTO BRINDADO:
131
FUNDACIÓN AZARALAGARTIJA DE LAS DUNAS
En el último estudio desarrollado des -
de la Sección de Herpetología de la FC -
NyM (UNLP), se evaluaron por primera 
vez la dinámica y riesgo de extinción de todas las subpoblaciones de esta especie que habitan en las dunas costeras pam-peanas, a partir de información nueva e investigaciones previas (Kacoliris et al., 2019). Como resultado de este estudio se definió el estado de conservación en base al tamaño de las subpoblaciones (asociado al tamaño del hábitat), el nivel de conectividad con subpoblaciones ve -
cinas y el grado de amenaza en relación al impacto combinado entre la población local y la p

In [ ]:
# Prompt:
prompt = 'cual es el correo electronico de un experto en Gestión de Recursos de la provincia de Buenos Aires?'

responder(prompt)

------------------------------------------------------------------------
PREGUNTA:
cual es el correo electronico de un experto en Gestión de Recursos de la provincia de Buenos Aires?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos vectorial
------------------------------------------------------------------------
CONTEXTO BRINDADO:
numero_de_certificado: 273
apellido: Argüeso
nombre: Jorge Amaru
provincia: Ciudad Autonoma de Buenos Aires
localidad: Ciudad Autonoma de Buenos Aires
correo_electronico: 273_ciudadautonomadebuenosaires@gmail.com
titulo_universitario: Licenciatura en Gestión de Recursos Naturales
------------------------------------------------------------------------
RESPUESTA:
El correo electrónico de un experto en Gestión de Recursos de la provincia de Buenos Aires es 273_ciudadautonomadebuenosaires@gmail.com.


In [ ]:
# Prompt:
prompt = '¿Cuales son las reservas naturales de la provincia de Buenos Aires?'

responder(prompt)

<ipython-input-27-3f617fdaeaeb>:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


------------------------------------------------------------------------
PREGUNTA:
¿Cuales son las reservas naturales de la provincia de Buenos Aires?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos grafos
------------------------------------------------------------------------
CONTEXTO BRINDADO:
Provincia: Buenos Aires Información de la reserva: creación: 2009, nombre: r"Campo Mar Chiquita - Dragones de Malvinas", hectáreas: 1700Provincia: Buenos Aires Información de la reserva: creación: 2013, nombre: r"Baterías - Charles Darwin", hectáreas: 1000Provincia: Buenos Aires Información de la reserva: creación: 2021, nombre: r"Faro Querandí", hectáreas: 44Provincia: Buenos Aires Información de la reserva: creación: 2023, nombre: r"Cerro Largo", hectáreas: 60Provincia: Buenos Aires Información de la reserva: creación: 2023, nombre: r"Faro San Antonio", hectáreas: 418
---------------------------------------------------------------------

In [ ]:
# Prompt:
prompt = '¿en donde vive la mariposa bandera argentina?'

responder(prompt)

ERROR:neo4j.io:Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
ERROR:neo4j.io:Failed to write data to connection IPv4Address(('748c956d.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))


------------------------------------------------------------------------
PREGUNTA:
¿en donde vive la mariposa bandera argentina?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos vectorial
------------------------------------------------------------------------
CONTEXTO BRINDADO:
71
FUNDACIÓN AZARAquien considere como instrumento de la 
Providencia esta aparición peregrina que busca siempre los lugares de la profunda sombra y algunas veces se extravía has -
ta el interior de las habitaciones huma-nas, visitando allí las próximas sombras y anunciándoles su breve partida á la tumba!” (Holmberg, 1887-1889). Como Döering describe inequívocamente a la bandera argentina no caben dudas que esa creencia no es una confusión con la mariposa nocturna conocida también como “mariposa de la muerte” o taparaco (Ascalapha odorata) (Bertonatti, 2013).
Fernando Bourquin (1945), quien rea-
lizó repetidas observaciones desde su quinta (“La Tacuartia”) 

In [ ]:
# Prompt:
prompt = '¿cuales son los rasgos etoecologicos del pajaro El bailarín castaño (Piprites pileata)?'

responder(prompt)

------------------------------------------------------------------------
PREGUNTA:
¿cuales son los rasgos etoecologicos del pajaro El bailarín castaño (Piprites pileata)?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos vectorial
------------------------------------------------------------------------
CONTEXTO BRINDADO:
concelos y D’Angelo Neto, 2009) y en San Pablo se lo registró moviéndose rá-pidamente en la copa de las araucarias y también formando parte de bandadas mixtas junto con el coludito de los pinos (Leptasthenura setaria) y el curutié pálido (Cranioleuca pallida) (Barbosa, 1992). En Misiones, en el PP Caá Yarí, se observó a un macho de bailarín integrando un ban-do mixto compuesto por el picolezna es -
triado (Heliobletus contaminatus), la mos -
queta media luna (Phylloscartes eximius) y el ticotico grande (Philydor rufus) (Maders et al., 2007) y en la RNC Papel Misione -
ro, también se observó un macho en un bando mixt

In [ ]:
# Prompt:
prompt = '¿Cuantas reservas naturales hay en Santa Fe?'

responder(prompt)

<ipython-input-27-3f617fdaeaeb>:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


------------------------------------------------------------------------
PREGUNTA:
¿Cuantas reservas naturales hay en Santa Fe?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos grafos
------------------------------------------------------------------------
CONTEXTO BRINDADO:
Provincia: Santa Fe Información de la reserva: creación: 2012, nombre: r"Campo Garabato", hectáreas: 2654Provincia: Santa Fe Información de la reserva: creación: 2012, nombre: r"Isla El Tala", hectáreas: 2000
------------------------------------------------------------------------
RESPUESTA:
En base a la información proporcionada, en la provincia de Santa Fe hay dos reservas naturales: "Campo Garabato" con 2654 hectáreas y "Isla El Tala" con 2000 hectáreas.
